<a href="https://colab.research.google.com/github/AV-016/peerdeck/blob/main/peerdeck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install streamlit pyngrok

In [45]:
%%writefile peerdeck.py
# Paste the entire Streamlit code from my previous response here
import streamlit as st
import random
import re

# [Rest of the code you received, starting with the in-memory data structures and ending with the footer note]

Overwriting peerdeck.py


In [46]:
!pip install streamlit pyngrok -q

In [47]:
from pyngrok import ngrok
!ngrok authtoken 31NlBCDsp7dbRaeMi49JkFxc9gh_3T69Jh2WGWbiGtezKJk5Z # Replace with your actual ngrok authtoken

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [42]:
import streamlit as st
import random
import re
import pandas as pd

if 'initialized' not in st.session_state:
    st.session_state.initialized = True

    # Locations
    local_locations = ['University A', 'University B', 'University C', 'University D']

    # Dummy users
    st.session_state.users = {}
    for i in range(1, 21):  # 20 students
        user_id = f'student{i}'
        location = random.choice(local_locations)
        st.session_state.users[user_id] = {
            'name': f'Student {i}',
            'type': 'student',
            'location': location,
            'points': random.randint(0, 200),
            'projects': []
        }
    for i in range(1, 6):  # 5 recruiters
        user_id = f'recruiter{i}'
        st.session_state.users[user_id] = {
            'name': f'Recruiter {i}',
            'type': 'recruiter',
            'location': 'Global',
            'points': 0,
            'projects': []
        }
    for i in range(1, 4):  # 3 NGOs
        user_id = f'ngo{i}'
        st.session_state.users[user_id] = {
            'name': f'NGO {i}',
            'type': 'ngo',
            'location': 'Global',
            'points': 0,
            'projects': []
        }

    # Dummy skills
    skills_list = ["Python", "Machine Learning", "Teamwork", "Aerospace Engineering", "Data Analysis", "Environmental Science", "Coding", "Project Management", "Web Development", "AI"]

    # Dummy projects (~30)
    st.session_state.projects = []
    st.session_state.project_id_counter = 1
    dummy_titles = ["AI Chatbot", "Solar Panel Design", "Web App for Education", "Data Analytics Dashboard", "Rocket Simulation", "Eco-Friendly Packaging", "Mobile Game", "Blockchain Wallet", "VR Training Tool", "Health Monitoring App"]
    dummy_descriptions = [
        "Built an AI-powered chatbot using Python and NLTK.",
        "Designed efficient solar panels for remote areas.",
        "Developed a web application to aid student learning.",
        "Created a dashboard for visualizing sales data.",
        "Simulated rocket trajectories using physics engines.",
        "Invented biodegradable packaging materials.",
        "Made a fun mobile game with Unity.",
        "Implemented a secure blockchain-based wallet.",
        "Created VR tools for employee training.",
        "App to monitor health metrics in real-time."
    ]
    for i in range(30):
        owner = random.choice([u for u in st.session_state.users if st.session_state.users[u]['type'] in ['student', 'ngo']])
        title = random.choice(dummy_titles) + f" {i+1}"
        desc = random.choice(dummy_descriptions)
        skills = random.sample(skills_list, random.randint(2,5))
        p_type = random.choice(['student', 'collab', 'ngo'])
        location = st.session_state.users[owner]['location']
        new_project = {
            'id': st.session_state.project_id_counter,
            'title': title,
            'description': desc,
            'skills': skills,
            'owner': owner,
            'collaborators': [],
            'type': p_type,
            'location': location,
            'is_ai_filtered': False,
            'likes': random.randint(0, 50),
            'image': None  # Placeholder for image
        }
        st.session_state.projects.append(new_project)
        st.session_state.users[owner]['projects'].append(st.session_state.project_id_counter)
        st.session_state.project_id_counter += 1

# Simple AI content detector
def is_ai_generated(text):
    ai_patterns = [r'\b(as an AI|generated by AI|large language model)\b', r'^Hello,.*\.$']
    if len(text.split()) > 50 or any(re.search(pattern, text, re.IGNORECASE) for pattern in ai_patterns):
        return True
    return False

# Sidebar for user selection
st.sidebar.title("PeerDeck MVP Prototype")
current_user = st.sidebar.selectbox("Select User", list(st.session_state.users.keys()))
user_info = st.session_state.users[current_user]

st.sidebar.write(f"Welcome, {user_info['name']}")
st.sidebar.write(f"Points: {user_info['points']}")
st.sidebar.write(f"Location: {user_info['location']}")

# Global toggle in sidebar for general use
mode = st.sidebar.radio("View Mode (for Feed & NGO)", ["Local", "Global"])

def filter_by_mode(items, key='location'):
    if mode == "Local":
        return [item for item in items if item.get(key) == user_info['location']]
    return items

# Main tabs
tab_names = ["Dashboard", "My Profile & Projects", "Project Feed", "Project Zone", "NGO Projects", "Recruiter Search", "Leaderboard"]
tabs = st.tabs(tab_names)

with tabs[0]:  # Dashboard
    st.header("Dashboard")
    st.write("Overview of your activity and recommendations.")
    col1, col2, col3 = st.columns(3)
    with col1:
        st.subheader("Your Points")
        st.metric("Points", user_info['points'])
    with col2:
        st.subheader("Your Projects")
        st.metric("Count", len(user_info['projects']))
    with col3:
        st.subheader("Collaborations")
        collab_count = sum(1 for p in st.session_state.projects if current_user in p['collaborators'])
        st.metric("Count", collab_count)
    # Recommended projects
    st.subheader("Recommended Projects for You")
    user_skills = set()
    for pid in user_info['projects']:
        proj = next((p for p in st.session_state.projects if p['id'] == pid), None)
        if proj:
            user_skills.update(proj['skills'])
    rec_projects = [p for p in st.session_state.projects if set(p['skills']) & user_skills and p['owner'] != current_user]
    rec_projects = random.sample(rec_projects, min(5, len(rec_projects))) if rec_projects else []
    for proj in rec_projects:
        st.write(f"- {proj['title']} by {st.session_state.users[proj['owner']]['name']} (Matching Skills: {', '.join(set(proj['skills']) & user_skills)})")

with tabs[1]:  # My Profile & Projects
    st.header("My Profile: Add Projects & Skills")
    st.write("Showcase your projects and skills here.")

    project_title = st.text_input("Project Title")
    project_desc = st.text_area("Description")
    skills = st.multiselect("Skills Gained", skills_list)
    project_type = st.selectbox("Project Type", ["Personal", "Open for Collaboration", "NGO"])
    uploaded_file = st.file_uploader("Upload Project Image", type=["jpg", "png", "jpeg"])

    if st.button("Add Project"):
        if is_ai_generated(project_desc):
            st.error("AI-generated content detected! Please write authentically.")
        else:
            image_data = uploaded_file.read() if uploaded_file else None
            p_type_map = {'Personal': 'student', 'Open for Collaboration': 'collab', 'NGO': 'ngo'}
            p_type = p_type_map[project_type]
            new_project = {
                'id': st.session_state.project_id_counter,
                'title': project_title,
                'description': project_desc,
                'skills': skills,
                'owner': current_user,
                'collaborators': [],
                'type': p_type,
                'location': user_info['location'] if p_type != 'ngo' else 'Global',
                'is_ai_filtered': False,
                'likes': 0,
                'image': image_data
            }
            st.session_state.projects.append(new_project)
            user_info['projects'].append(st.session_state.project_id_counter)
            user_info['points'] += 20
            st.session_state.project_id_counter += 1
            st.success("Project added! +20 points")

    st.subheader("Your Projects")
    for pid in user_info['projects']:
        proj = next((p for p in st.session_state.projects if p['id'] == pid), None)
        if proj:
            with st.expander(proj['title']):
                if proj['image']:
                    st.image(proj['image'], use_column_width=True)
                st.write(proj['description'])
                st.write(f"Skills: {', '.join(proj['skills'])}")
                st.write(f"Likes: {proj['likes']}")

with tabs[2]:  # Project Feed
    st.header("Project Feed")
    st.write("Browse projects in your feed. AI-filtered content is hidden.")
    filtered_projects = filter_by_mode([p for p in st.session_state.projects if not p['is_ai_filtered']])
    for proj in filtered_projects:
        owner_name = st.session_state.users[proj['owner']]['name']
        with st.expander(f"{proj['title']} by {owner_name}"):
            if proj['image']:
                st.image(proj['image'], use_column_width=True)
            st.write(proj['description'])
            st.write(f"Skills: {', '.join(proj['skills'])}")
            st.write(f"Likes: {proj['likes']}")
            if st.button(f"Like {proj['title']}", key=f"like_feed_{proj['id']}"):
                proj['likes'] += 1
                user_info['points'] += 1
                st.success("Liked! +1 point")
                st.rerun()

with tabs[3]:  # Project Zone
    st.header("Project Zone: Collaborate")
    st.write("Find and join interdisciplinary projects.")
    # Specific toggle for Project Zone
    zone_mode = st.radio("Project Zone View Mode", ["Local", "Global"], horizontal=True)
    def zone_filter(items):
        if zone_mode == "Local":
            return [item for item in items if item['location'] == user_info['location']]
        return items
    collab_projects = zone_filter([p for p in st.session_state.projects if p['type'] == 'collab'])
    for proj in collab_projects:
        owner_name = st.session_state.users[proj['owner']]['name']
        with st.expander(f"{proj['title']} by {owner_name}"):
            if proj['image']:
                st.image(proj['image'], use_column_width=True)
            st.write(proj['description'])
            st.write(f"Skills needed: {', '.join(proj['skills'])}")
            if current_user != proj['owner'] and current_user not in proj['collaborators']:
                if st.button(f"Join {proj['title']}", key=f"join_{proj['id']}"):
                    proj['collaborators'].append(current_user)
                    user_info['points'] += 10
                    st.success("Joined project! +10 points")
                    st.rerun()
            st.write(f"Collaborators: {', '.join([st.session_state.users[c]['name'] for c in proj['collaborators']])}")

with tabs[4]:  # NGO Projects
    st.header("NGO Projects: Gain Real-World Experience")
    st.write("NGOs post tasks; join for points and experience.")

    if user_info['type'] == 'ngo':
        ngo_title = st.text_input("NGO Project Title")
        ngo_desc = st.text_area("Task Description")
        ngo_skills = st.multiselect("Required Skills", skills_list)
        uploaded_file_ngo = st.file_uploader("Upload Image for NGO Project", type=["jpg", "png", "jpeg"])

        if st.button("Post NGO Project"):
            if is_ai_generated(ngo_desc):
                st.error("AI-generated content detected! Please write authentically.")
            else:
                image_data = uploaded_file_ngo.read() if uploaded_file_ngo else None
                new_ngo = {
                    'id': st.session_state.project_id_counter,
                    'title': ngo_title,
                    'description': ngo_desc,
                    'skills': ngo_skills,
                    'owner': current_user,
                    'collaborators': [],
                    'type': 'ngo',
                    'location': 'Global',
                    'is_ai_filtered': False,
                    'likes': 0,
                    'image': image_data
                }
                st.session_state.projects.append(new_ngo)
                user_info['projects'].append(st.session_state.project_id_counter)
                st.session_state.project_id_counter += 1
                st.success("NGO project posted!")

    ngo_projects = filter_by_mode([p for p in st.session_state.projects if p['type'] == 'ngo'])
    for proj in ngo_projects:
        owner_name = st.session_state.users[proj['owner']]['name']
        with st.expander(f"{proj['title']} by {owner_name}"):
            if proj['image']:
                st.image(proj['image'], use_column_width=True)
            st.write(proj['description'])
            st.write(f"Skills: {', '.join(proj['skills'])}")
            if current_user != proj['owner'] and current_user not in proj['collaborators']:
                if st.button(f"Take on {proj['title']}", key=f"take_{proj['id']}"):
                    proj['collaborators'].append(current_user)
                    user_info['points'] += 30
                    st.success("Task taken! +30 points")
                    st.rerun()
            st.write(f"Team: {', '.join([st.session_state.users[c]['name'] for c in proj['collaborators']])}")

with tabs[5]:  # Recruiter Search
    st.header("Recruiter Search: Skill-Based Recommendations")
    st.write("Search for skills to find matching students.")

    if user_info['type'] == 'recruiter':
        search_skill = st.text_input("Search by Skill")
        if search_skill:
            matching_projects = filter_by_mode([p for p in st.session_state.projects if search_skill.lower() in [s.lower() for s in p['skills']]])
            if matching_projects:
                st.write("Recommended Profiles:")
                seen_users = set()
                for proj in matching_projects:
                    owner = proj['owner']
                    if owner not in seen_users:
                        st.write(f"- **{st.session_state.users[owner]['name']}** (Location: {st.session_state.users[owner]['location']}, Points: {st.session_state.users[owner]['points']})")
                        st.write(f"  Project: {proj['title']} (Skills: {', '.join(proj['skills'])})")
                        seen_users.add(owner)
            else:
                st.write("No matches found.")
    else:
        st.write("This view is for recruiters only.")

with tabs[6]:  # Leaderboard
    st.header("Leaderboard")
    st.write("Top users by points.")
    lb_mode = st.radio("Leaderboard Mode", ["Local", "Global"], horizontal=True)
    if lb_mode == "Local":
        local_users = {u: info for u, info in st.session_state.users.items() if info['type'] == 'student' and info['location'] == user_info['location']}
        if local_users:
            local_df = pd.DataFrame([(info['name'], info['points']) for info in local_users.values()], columns=['Name', 'Points'])
            local_df = local_df.sort_values('Points', ascending=False).reset_index(drop=True)
            st.table(local_df)
        else:
            st.write("No local users found.")
    else:
        global_users = {u: info for u, info in st.session_state.users.items() if info['type'] == 'student'}
        global_df = pd.DataFrame([(info['name'], info['points']) for info in global_users.values()], columns=['Name', 'Points'])
        global_df = global_df.sort_values('Points', ascending=False).reset_index(drop=True)
        st.table(global_df.head(20))  # Top 20 global

# Footer
st.write("---")
st.write("This is an enhanced MVP prototype ready for Streamlit deployment. Data persists in session state. Features include dummy data, image uploads, likes, dashboard, and mode-specific leaderboards.")

2025-08-16 19:26:03.158 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 19:26:03.161 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 19:26:03.162 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 19:26:03.164 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 19:26:03.165 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 19:26:03.166 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 19:26:03.167 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 19:26:03.167 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [48]:
from pyngrok import ngrok
import subprocess

# Replace with your NEW ngrok authtoken (after revoking the exposed one)
!ngrok authtoken 31NlW0iMiPvDDZYO4rK7AUpfWvp_5Uu8BFssDfE7opRQbq1kN # Uncomment this line and replace with your new authtoken
# Start Streamlit in the background
get_ipython().system_raw('streamlit run peerdeck.py &')

# Create a tunnel to Streamlit port 8501
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


PyngrokNgrokHTTPError: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: The authtoken credential '31NhhoPjiKLV3FPqMoWSUTpOasX' has been revoked\nand is no longer valid.\r\n\r\nERR_NGROK_300\r\n"}}
